# MedMNIST Classification with CNN
This notebook loads the MedMNIST dataset, builds a CNN model, trains it, evaluates it, and saves the model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
# Path to MedMNIST dataset
path = "/kaggle/input/medical-mnist"

# Load training dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    path,
    labels='inferred',
    batch_size=64,
    image_size=(32, 32),
    color_mode='grayscale',
    label_mode='int',
    validation_split=0.2,
    subset='training',
    seed=42
)

# Load validation dataset
val_dataset = tf.keras.utils.image_dataset_from_directory(
    path,
    labels='inferred',
    batch_size=64,
    image_size=(32, 32),
    color_mode='grayscale',
    label_mode='int',
    validation_split=0.2,
    subset='validation',
    seed=42
)

# Optimize dataset performance
train_dataset = train_dataset.cache().prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
# Class names mapping
class_names = {
    0: "AbdomenCT",
    1: "BreastMRI",
    2: "CXR",
    3: "ChestCT",
    4: "Hand",
    5: "HeadCT"
}

# Visualize a random sample
size = len(list(train_dataset.take(1).unbatch()))
rand_index = np.random.randint(0, size-1)

for img_train, label_train in train_dataset.take(1):
    plt.imshow(img_train[rand_index], cmap='gray')
    plt.title(f"Label: {class_names[label_train[rand_index].numpy()]}")
    plt.axis('off')
    plt.show()

In [ ]:
def build_CNN_Model():
    inputs = tf.keras.layers.Input((32, 32, 1))
    x = tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)
    
    x = tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)
    
    x = tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)
    
    x = tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)
    
    x = tf.keras.layers.Flatten()(x)
    outputs = tf.keras.layers.Dense(6, activation='softmax')(x)
    
    return tf.keras.models.Model(inputs, outputs)

model = build_CNN_Model()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

In [ ]:
loss, acc = model.evaluate(val_dataset)
print(f"Validation Accuracy: {acc:.4f}, Loss: {loss:.4f}")

In [ ]:
model.save("Medical_Image_Classification.keras")